# Primeros pasos

In [1]:
# Importamos las librerías necesarias para el proyecto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

In [89]:
# Indicar la ruta del directorio 'lib'
lib_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'lib'))
if lib_path not in sys.path:
    sys.path.append(lib_path)

# Importar todas las funciones 
from functions import *

# Configurar la carga automática de los cambios realizados en funciones
%reload_ext autoreload
%autoreload 2

## Cargar los datos

In [3]:
# Cargamos la base de datos principal
df = pd.read_csv('../data/indicadores_generales_distritos.csv', sep=';', encoding='utf-8-sig')

In [4]:
# Configuramos la visualización del dataframe para mostrar todas las columnas
pd.set_option('display.max_columns', None)
df

,Orden,Periodo panel,ciudad,cod_distrito,distrito,cod_barrio,barrio,año,fecha_indicador,fuente_indicador,categoría_1,categoría_2,indicador_nivel1,indicador_nivel2,indicador_nivel3,unidad_indicador,indicador_completo,valor_indicador
0,1,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Características Generales del Distrito-Barrio,NaN,Superficie,NaN,NaN,Ha,Superficie (Ha.),60.446
1,2,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Características Generales del Distrito-Barrio,NaN,Densidad de población,NaN,NaN,Habitantes/Ha.,Población densidad (hab./Ha.),54
2,3,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,NaN,NaN,Habitantes,Número Habitantes,3.266.126
3,4,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,Hombres,NaN,Habitantes,Población Hombres,1.521.178
4,5,2020,Ciudad de Madrid,NaN,NaN,NaN,NaN,2019,01/01/2019,Estadística Ayuntamiento de Madrid,Población del distrito,Estructura de población,Número de Habitantes,Mujeres,NaN,Habitantes,Población Mujeres,1.744.948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69540,69541,2023,Ciudad de Madrid,21.0,Barajas,211.0,Alameda de Osuna,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"126,39"
69541,69542,2023,Ciudad de Madrid,21.0,Barajas,212.0,Aeropuerto,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"63,81"
69542,69543,2023,Ciudad de Madrid,21.0,Barajas,213.0,Casco Histórico de Barajas,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"96,15"
69543,69544,2023,Ciudad de Madrid,21.0,Barajas,214.0,Timón,2022,01/01/2022,Estadística Ayuntamiento de Madrid,Vivienda,NaN,Inmuebles,Uso residencial,Superficie media,m2,Superficie media construida (m2) inmuebles de ...,"128,07"


## Limpieza y revisión de la base de datos principal

### Limpieza de datos

In [5]:
# Chequeamos el tipo de datos de las columnas y los valores nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69545 entries, 0 to 69544
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Orden               69545 non-null  int64  
 1   Periodo panel       69545 non-null  int64  
 2   ciudad              69545 non-null  object 
 3   cod_distrito        68464 non-null  float64
 4   distrito            68464 non-null  object 
 5   cod_barrio          50804 non-null  float64
 6   barrio              50812 non-null  object 
 7   año                 69545 non-null  int64  
 8   fecha_indicador     69545 non-null  object 
 9   fuente_indicador    69545 non-null  object 
 10  categoría_1         69545 non-null  object 
 11  categoría_2         61903 non-null  object 
 12  indicador_nivel1    69545 non-null  object 
 13  indicador_nivel2    60506 non-null  object 
 14  indicador_nivel3    26899 non-null  object 
 15  unidad_indicador    69526 non-null  object 
 16  indi

In [6]:
# Comprobamos que no haya duplicados
df.duplicated().sum()

np.int64(0)

In [7]:
df.nunique()

Orden                 69545
Periodo panel             4
ciudad                    1
cod_distrito             21
distrito                 32
cod_barrio              131
barrio                  206
año                      12
fecha_indicador          19
fuente_indicador         14
categoría_1              19
categoría_2              51
indicador_nivel1         44
indicador_nivel2        204
indicador_nivel3         78
unidad_indicador         39
indicador_completo      410
valor_indicador       19492
dtype: int64

In [8]:
# Eliminar espacios al comienzo del nombre de algunos distritos
eliminar_espacios(df, 'distrito')
#Eliminar espacios adicionales de los indicadores
eliminar_espacios(df, 'indicador_completo')

In [9]:
# Adaptamos el formato de puntuación en los valores para una correcta interpretación en la conversión a números
estandarizar_numeros(df, 'valor_indicador')

In [10]:
# Convertir a numérico el valor de los indicadores
convertir_a_numerico(df,'valor_indicador')

### Explorar el contenido

In [11]:
# Visualizamos las categorías generales y más específicas
df.groupby('categoría_1')['categoría_2'].value_counts()

categoría_1                             categoría_2                                              
Calidad de vida                         Satisfacción con espacios públicos                             572
                                        Calidad de vida y agenda pública                               264
                                        Percepción de la seguridad                                     264
                                        Principales poblemas de la ciudadanía                          264
                                        Problemas que afectan a la seguridad                           166
                                        Administración de la ciudad                                    154
                                        Amigabilidad de la ciudad                                       88
                                        Ciudadanía                                                      44
Educación                               Poblac

In [12]:
#Lista de todos los indicadores que están en esta base de datos
df['indicador_completo'].unique()

array(['Superficie (Ha.)', 'Población densidad (hab./Ha.)',
       'Número Habitantes', 'Población Hombres', 'Población Mujeres',
       'Edad media de la población', 'Población de 0 a 14 años',
       'Población de 15 a 29 años', 'Población de 30 a 44  años',
       'Población de 45 a 64 años', 'Población de 65 a 79 años',
       'Población de 80 años y más', 'Población de 65 años y más',
       'Población en etapa educativa (Población de 0 a 16 años -16 no incluidos)',
       'Proporción de envejecimiento (Población mayor de 65 años/Población total)',
       'Proporción de sobre-envejecimiento (Población mayor de 80 años/ Población mayor de 65 años)',
       'Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64)',
       'Personas con nacionalidad española',
       'Personas con nacionalidad española Hombres',
       'Personas con nacionalidad española Mujeres',
       'Personas con nacionalidad extranjera',
       'Personas con nacionalidad extranjera Homb

# Extraer datos por distrito

### Guía general

#### **Ámbitos de desigualdad e indicadores**

#### 1. Población (df_poblacion)
Este conjunto de datos refleja las características demográficas de cada distrito.

- **Densidad de población (hab./Ha.):** La cantidad de habitantes por hectárea, refleja la concentración de la población en un área específica.
- **Número de habitantes:** El total de personas que viven en el distrito.
- **Edad media de la población:** La media de edad de las personas que habitan en el distrito.
- **Proporción de envejecimiento:** Porcentaje de personas mayores (usualmente de 65 años o más) respecto al total de habitantes.
- **Proporción de personas migrantes:** Porcentaje de población nacida fuera de España, importante para entender la diversidad y posibles necesidades sociales del distrito.
- **Índice de dependencia:** Relación entre la población dependiente (menores de 16 años y mayores de 64) y la población en edad laboral (16-64 años).

#### 2. Salud (df_salud)
Indicadores sobre la salud y el acceso a servicios sanitarios en cada distrito.

- **Tasa de centros sanitarios (por cada 1000 habitantes)**: El número de centros de salud por cada 1000 habitantes.
- **Probabilidad de padecer enfermedad mental**: Estimación basada en la prevalencia de enfermedades mentales en el distrito.
- **Autopercepción de buen estado de salud (%)**: Proporción de personas que califican su salud como "buena" o "muy buena".
- **Sedentarismo (%)**: Proporción de la población que no realiza actividad física de forma regular.
- **Presencia de enfermedad crónica (%)**: Proporción de personas con enfermedades crónicas en el distrito.
- **Esperanza de vida media**: La media de años que se espera que viva una persona en el distrito.

#### 3. Bienestar Social (df_bienestar)
Estos indicadores reflejan la percepción de bienestar y seguridad en cada distrito.

- **Calidad de vida (%)**: Proporción de personas que perciben su calidad de vida en el barrio como buena o muy buena.
-**Percepción de seguridad (%)**: Porcentaje de personas que se sienten seguras en su distrito.
-**Satisfacción de vivir en su barrio (%)**: Grado de satisfacción de los residentes con respecto a su lugar de residencia.
-**Tasa de intervenciones policiales (por cada 1000 habitantes)**: Número de intervenciones de la policía en el distrito, relacionadas con delitos.

#### 4. Economía y Empleo (df_economia)
Indicadores que reflejan la situación económica y laboral en cada distrito.

- **Renta media disponible (€)**: Ingreso medio disponible por persona en el distrito.
- **Tasa de paro (%)**: Porcentaje de la población activa que está desempleada.
- **Tasa de paro de larga duración (%)**: Porcentaje de desempleados que han estado sin trabajo durante un periodo prolongado (más de 12 meses).
- **Tasa de paro juvenil (%)**: Porcentaje de personas entre 16 y 24 años que están desempleadas.
- **Pensión media (€)**: Promedio de la pensión que reciben los jubilados en el distrito.
- **Tasa de comercios (por cada 1000 habitantes)**: Cantidad de comercios en relación con la población.

#### 5. Educación (df_educacion)
Indicadores educativos que reflejan el acceso y el nivel educativo de la población.

- **Tasa de centros educativos (por cada 1000 habitantes)**: Número de centros educativos por cada 1000 habitantes.
- **Tasa de centros públicos de enseñanza obligatoria (por cada 1000 habitantes)**: Relación de centros públicos (desde infantil hasta secundaria) respecto al total de centros educativos en el distrito.
- **Tasa de absentismo escolar (por cada 1000 habitantes)**: Proporción de estudiantes que no asisten a la escuela.
- **Tasa de analfabetismo (por cada 1000 habitantes)**: Proporción de personas mayores de 25 años que no saben leer ni escribir.
- **Tasa de población con solo educación obligatoria (por cada 1000 habitantes)**: Proporción de la población con estudios hasta el nivel de la ESO o equivalente.
- **Tasa de población con estudios superiores (por cada 1000 habitantes)**: Proporción de personas con estudios universitarios o equivalentes.

#### 6. Cultura, Deporte y Ocio (df_cultura)
Indicadores sobre la infraestructura cultural y recreativa, así como la satisfacción de los ciudadanos.

- **Tasa de bibliotecas (por cada 10000 habitantes)**: Cantidad de bibliotecas públicas en el distrito.
- Tasa de centros culturales (por cada 10000 habitantes): Número de centros culturales en relación con la población.
- Tasa de zonas verdes (por cada 10000 habitantes): Superficie de zonas verdes y parques por cada 10000 habitantes.
- **Satisfacción con los espacios verdes (0-10)**: Grado de satisfacción de los ciudadanos con las zonas verdes del distrito.
- **Tasa de superficie deportiva (m² por cada 10000 habitantes)**: Espacio disponible para instalaciones deportivas por cada 10000 habitantes.
- **Grado de satisfacción con instalaciones deportivas (0-10)**: Satisfacción de los residentes con las instalaciones deportivas.
- **Grado de satisfacción con fiestas y eventos (0-10)**: Opinión de los ciudadanos respecto a la calidad de los eventos culturales y fiestas.

#### 7. Servicios Sociales (df_social)
Indicadores que reflejan el acceso a los servicios sociales en cada distrito.

- **Tasa de centros de servicios sociales (por cada 1000 habitantes)**: Número de centros de servicios sociales por cada 1000 habitantes.
- **Tasa de demandas de intervención en los Centros de Atención a la Infancia (CAI)**: Proporción de demandas de atención a la infancia.
- **Tasa de personas atendidas en los servicios sociales (por cada 1000 habitantes)**: Proporción de personas que reciben atención social.
- **Tasa de ayuda a domicilio**: Porcentaje de personas que reciben servicios de ayuda a domicilio.
- **Tasa de residencias para mayores (por cada 1000 habitantes)**: Número de residencias de ancianos en relación con la población.

#### 8. Presupuesto (df_presupuestos)
Información sobre la inversión pública en cada distrito y área de gasto.

- **Área de inversión**: Cultura, deportes, infraestructuras, etc.
- **Total invertido por área y año**: Inversión total realizada en euros

## Dataframe población

### Extraer información del dataframe principal

In [13]:
# Crear un dataframe con los indicadores de población por distrito
poblacion_indicadores = [
    'Población densidad (hab./Ha.)', 
    'Número Habitantes', 
    'Edad media de la población', 
    'Proporción de envejecimiento (Población mayor de 65 años/Población total)', 
    'Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64)',
    'Proporción de personas migrantes (Población extranjera menos UE y resto países de OCDE / Población total)']

# Filtramos el dataframe original solo con los indicadores relevantes y agrupamos por distrito
df_poblacion = df[df['indicador_completo'].isin(poblacion_indicadores)].pivot_table(
    index=['cod_distrito', 'distrito'], 
    columns='indicador_completo', 
    values='valor_indicador', 
    aggfunc='first').reset_index()

### Limpieza

In [14]:
df_poblacion.rename(columns={
    'Población densidad (hab./Ha.)': 'densidad_poblacion',
    'Edad media de la población': 'edad_media',
    'Proporción de envejecimiento (Población mayor de 65 años/Población total)':'proporcion_envejecimiento',
    'Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64)': 'indice_dependencia',
    'Proporción de personas migrantes (Población extranjera menos UE y resto países de OCDE / Población total)': 'proporcion_migrantes'}, inplace=True)

estandarizar_columnas(df_poblacion)

### Resultado

In [15]:
# Mostrar el dataframe de población por distritos
df_poblacion

indicador_completo,cod_distrito,distrito,edad_media,numero_habitantes,densidad_poblacion,proporcion_envejecimiento,proporcion_migrantes,indice_dependencia
0,1.0,Centro,44.03,134881.0,258.0,16.05,12.54,32.15
1,2.0,Arganzuela,44.55,153830.0,238.0,19.27,6.50,46.81
2,3.0,Retiro,47.04,119379.0,218.0,25.76,4.31,62.09
3,4.0,Salamanca,46.24,146148.0,271.0,23.89,7.14,54.41
4,5.0,Chamartín,45.39,145865.0,159.0,23.28,5.49,58.97
5,6.0,Tetuán,44.13,157937.0,294.0,19.33,14.96,45.03
6,7.0,Chamberí,46.34,139448.0,298.0,24.18,6.19,53.97
7,8.0,Fuencarral-El Pardo,42.87,246021.0,10.0,20.80,5.58,63.31
8,9.0,Moncloa-Aravaca,44.57,119423.0,26.0,21.90,6.50,57.28
9,10.0,Latina,46.49,238154.0,94.0,24.79,11.50,59.30


## Dataframe salud

### Extraer información del dataframe principal

In [16]:
# Indicadores de la categoría Salud
salud_indicadores = [
    'Número Habitantes',
    'Esperanza de vida al nacer Mujeres', 
    'Esperanza de vida al nacer Hombres',
    'Autopercepción de buen estado de salud  (porcentaje respuesta muy buena + buena)',
    'Sedentarismo', 
    'Índice de obesidad',
    'Consumo de medicamentos',  
    'Presencia de enfermedad crónica', 
    'Probabilidad de padecer enfermedad mental (GHQ-12)          (2018. EMS)']

# Crear dataframe de Salud por distritos
df_salud = df[df['indicador_completo'].isin(salud_indicadores)].pivot_table(
    index=['cod_distrito', 'distrito'], 
    columns='indicador_completo', 
    values='valor_indicador', 
    aggfunc='first').reset_index()

### Limpieza y preparación

In [17]:
# Simplificar el nombre de los indicadores
df_salud.rename(columns={
    'Autopercepción de buen estado de salud  (porcentaje respuesta muy buena + buena)': 'autopercepcion_salud_buena',
    'Probabilidad de padecer enfermedad mental (GHQ-12)          (2018. EMS)': 'probabilidad_enfermedad_mental',
    'Presencia de enfermedad crónica':'presencia_enfermedad_cronica'}, inplace=True)
# Estandarizar los nombres de las columnas
estandarizar_columnas(df_salud)

Calcular el índice de la esperanza de vida por distritos (sin diferenciar por géneros)

In [18]:
#Calcular la esperanza de vida media por distritos
df_salud['esperanza_vida'] = df_salud[['esperanza_de_vida_al_nacer_hombres', 'esperanza_de_vida_al_nacer_mujeres']].mean(axis=1)
df_salud.drop(['esperanza_de_vida_al_nacer_hombres', 'esperanza_de_vida_al_nacer_mujeres'], axis=1, inplace=True)

### Añadir información de los centros sanitarios

In [19]:
# Cargar la información de la base de datos
df_centros_salud = pd.read_csv('../data/centros-atencion-medica.csv', sep=';', encoding='latin1')

# Hacer la preparación necesaria de los datos para unir los dataframes
df_centros_salud.columns = df_centros_salud.columns.str.lower().str.replace('-', '_')
df_centros_salud = df_centros_salud[['nombre', 'cod_distrito']]
# Calcular el recuento de centros por distrito
df_centros_salud = df_centros_salud.groupby('cod_distrito').size().reset_index(name='recuento_centros')

### Unir la información

In [20]:
# Realizar el merge para tener el recuento de centros sanitarios en el dataframe de salud
df_salud = pd.merge(df_salud, df_centros_salud, on='cod_distrito', how='left')

# Calcular la tasa de centros por mil habitantes
df_salud['tasa_centros_salud'] = (df_salud['recuento_centros'] / df_salud['numero_habitantes']) * 1000

# Seleccionar las columnas finales
df_salud = df_salud[['cod_distrito', 'distrito', 'tasa_centros_salud', 
                     'probabilidad_enfermedad_mental',
                     'autopercepcion_salud_buena',
                     'sedentarismo', 'presencia_enfermedad_cronica', 'esperanza_vida']]

### Resultado

In [21]:
df_salud

,cod_distrito,distrito,tasa_centros_salud,probabilidad_enfermedad_mental,autopercepcion_salud_buena,sedentarismo,presencia_enfermedad_cronica,esperanza_vida
0,1.0,Centro,0.088967,0.22,0.76,0.27,0.34,84.5
1,2.0,Arganzuela,0.078008,0.17,0.74,0.26,0.43,85.0
2,3.0,Retiro,0.075390,0.18,0.77,0.24,0.40,86.0
3,4.0,Salamanca,0.143690,0.17,0.76,0.31,0.41,85.5
4,5.0,Chamartín,0.082268,0.18,0.78,0.25,0.35,85.5
5,6.0,Tetuán,0.069648,0.21,0.67,0.31,0.41,84.5
6,7.0,Chamberí,0.129080,0.21,0.76,0.28,0.39,84.5
7,8.0,Fuencarral-El Pardo,0.077229,0.21,0.73,0.29,0.37,85.5
8,9.0,Moncloa-Aravaca,0.159098,0.19,0.78,0.23,0.35,85.5
9,10.0,Latina,0.088178,0.23,0.71,0.30,0.44,85.0


## Dataframe bienestar social

### Extraer información del dataframe principal

In [22]:
# Indicadores de Bienestar Social
bienestar_indicadores = [
    'Número Habitantes', 
    'Satisfacción de vivir en su barrio', 
    'Calidad de vida actual en su barrio',
    'Percepción de seguridad en Madrid', 
    'Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas',
    'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas',
    'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio',
    'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas']

# Crear dataframe de Bienestar Social por distritos
df_bienestar = df[df['indicador_completo'].isin(bienestar_indicadores)].pivot_table(
    index=['cod_distrito', 'distrito'], 
    columns='indicador_completo', 
    values='valor_indicador', 
    aggfunc='first').reset_index()

df_bienestar

indicador_completo,cod_distrito,distrito,Calidad de vida actual en su barrio,Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas,Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas,Número Habitantes,Percepción de seguridad en Madrid,Satisfacción de vivir en su barrio
0,1.0,Centro,7.36,943.0,1439.0,716.0,2155.0,134881.0,7.65,7.73
1,2.0,Arganzuela,7.79,199.0,287.0,19.0,306.0,153830.0,6.71,8.13
2,3.0,Retiro,8.03,70.0,86.0,20.0,106.0,119379.0,6.50,8.22
3,4.0,Salamanca,7.75,113.0,438.0,18.0,456.0,146148.0,6.98,7.82
4,5.0,Chamartín,8.05,100.0,154.0,15.0,169.0,145865.0,7.49,8.40
5,6.0,Tetuán,6.71,344.0,228.0,30.0,258.0,157937.0,7.03,6.65
6,7.0,Chamberí,8.00,144.0,158.0,7.0,165.0,139448.0,6.77,7.91
7,8.0,Fuencarral-El Pardo,7.58,130.0,110.0,22.0,132.0,246021.0,6.17,7.36
8,9.0,Moncloa-Aravaca,7.95,174.0,276.0,20.0,296.0,119423.0,7.08,8.10
9,10.0,Latina,6.86,157.0,185.0,29.0,214.0,238154.0,7.30,6.85


### Limpieza y preparación

In [23]:
# Calcular la tasa de intervenciones policiales por cada 1.000 habitantes
df_bienestar['tasa_intervenciones_policia'] = ((
    (df_bienestar['Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas']+
    df_bienestar['Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas']+
    df_bienestar['Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio']+
    df_bienestar['Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas']) / 
    df_bienestar['Número Habitantes']) * 1000).round(2)


In [24]:
# Eliminar las columnas innecesarias
df_bienestar.drop(['Intervenciones de la Policía Municipal en materia de seguridad: delitos relacionados con las personas',
       'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con el patrimonio',
       'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia de armas',
       'Intervenciones de la Policía Municipal en materia de seguridad: relacionadas con la tenencia y consumo de drogas',
       'Número Habitantes'], axis=1, inplace=True)

In [25]:
# Simplificar los nombres de las columnas
df_bienestar.rename(columns={
    'Calidad de vida actual en su barrio': 'calidad_vida',
    'Madrid ciudad amigable con las personas lesbianas, gays, transexuales y bisexuales': 'amigable_lgbt',
    'Percepción de seguridad en Madrid':'percepcion_seguridad',
    'Satisfacción de la convivencia vecinal': 'satisfaccion_convivencia_distrito',
    'Satisfacción de vivir en su barrio': 'satisfaccion_vivir_distrito'}, inplace=True)
# Estandarizar los nombres de las columnas
estandarizar_columnas(df_bienestar)


### Resultado

In [26]:
df_bienestar

indicador_completo,cod_distrito,distrito,calidad_vida,percepcion_seguridad,satisfaccion_vivir_distrito,tasa_intervenciones_policia
0,1.0,Centro,7.36,7.65,7.73,38.95
1,2.0,Arganzuela,7.79,6.71,8.13,5.27
2,3.0,Retiro,8.03,6.50,8.22,2.36
3,4.0,Salamanca,7.75,6.98,7.82,7.01
4,5.0,Chamartín,8.05,7.49,8.40,3.00
5,6.0,Tetuán,6.71,7.03,6.65,5.45
6,7.0,Chamberí,8.00,6.77,7.91,3.40
7,8.0,Fuencarral-El Pardo,7.58,6.17,7.36,1.60
8,9.0,Moncloa-Aravaca,7.95,7.08,8.10,6.41
9,10.0,Latina,6.86,7.30,6.85,2.46


## Dataframe economía y empleo

### Extraer información del dataframe principal

In [27]:
# Indicadores de Economía y Empleo
economia_indicadores = [
    'Número Habitantes',
    'Renta disponible media por persona', 
    'Pensión media mensual Hombres', 
    'Pensión media mensual  Mujeres', 
    'Tasa absoluta de paro registrado (febrero)', 
    'Tasa de desempleo en mujeres de 16 a 24 años', 
    'Tasa de desempleo en hombres de 16 a 24 años', 
    'Personas paradas de larga duración (febrero)']

# Crear dataframe de Economía y Empleo por distritos
df_economia = df[df['indicador_completo'].isin(economia_indicadores)].pivot_table(
    index=['cod_distrito', 'distrito'], 
    columns='indicador_completo', 
    values='valor_indicador', 
    aggfunc='first').reset_index()

### Limpieza y preparación

In [28]:
# Simplificar los nombres de las columnas
df_economia.rename(columns={
    'Personas paradas de larga duración (febrero)': 'parados_larga_duracion',
    'Tasa absoluta de paro registrado (febrero)': 'tasa_paro',
    'Renta disponible media por persona': 'renta_media'}, inplace=True)
# Estandarizar los nombres de las columnas
estandarizar_columnas(df_economia)

In [29]:
# Calcular tasa de parados de larga duración (*100 para estar en la escala de la tasa de paro absoluta)
df_economia['tasa_paro_larga_duracion'] = ((df_economia['parados_larga_duracion'] / df_economia['numero_habitantes']) * 100).round(2)

# Calcular la media de la tasa de desempleo entre los jóvenes (sin diferenciar hombres de mujeres)
df_economia['tasa_paro_joven'] = df_economia[['tasa_de_desempleo_en_hombres_de_16_a_24_anos', 'tasa_de_desempleo_en_mujeres_de_16_a_24_anos']].mean(axis=1).round(2)

#Calcular la pension media
df_economia['pension_media'] = ((df_economia['pension_media_mensual_hombres'] + df_economia['pension_media_mensual__mujeres']) / 2).round(2)

#Eliminar columnas innecesarias
df_economia.drop(['parados_larga_duracion','tasa_de_desempleo_en_hombres_de_16_a_24_anos',
                  'tasa_de_desempleo_en_mujeres_de_16_a_24_anos', 'pension_media_mensual_hombres', 'pension_media_mensual__mujeres'], axis=1, inplace=True)

### Añadir información de comercios y empresas

In [30]:
df_comercio = pd.read_csv('../data/locales_madrid.csv')

# Realizar el merge para tener el recuento de locales activos
df_economia = pd.merge(df_economia, df_comercio, on='cod_distrito', how='left')

# Calcular la tasa de comercios por cada mil habitantes
df_economia['tasa_comercios'] = ((df_economia['num_locales'] / df_economia['numero_habitantes']) * 1000).round(2)

In [31]:
#Eliminamos columnas innecesarias
df_economia.drop(['num_locales', 'numero_habitantes'], axis=1, inplace=True)

### Resultado

In [32]:
df_economia

,cod_distrito,distrito,renta_media,tasa_paro,tasa_paro_larga_duracion,tasa_paro_joven,pension_media,tasa_comercios
0,1.0,Centro,24920.0,7.05,1.87,2.95,1151.0,81.20
1,2.0,Arganzuela,24511.0,6.39,1.59,3.08,1327.0,31.60
2,3.0,Retiro,27616.0,5.50,1.27,1.83,1455.5,29.21
3,4.0,Salamanca,28128.0,4.89,1.16,1.86,1438.0,50.53
4,5.0,Chamartín,28044.0,4.91,1.18,1.38,1421.5,39.64
5,6.0,Tetuán,21324.0,7.29,1.96,4.12,1183.5,39.17
6,7.0,Chamberí,27761.0,5.24,1.17,2.10,1458.5,49.18
7,8.0,Fuencarral-El Pardo,22765.0,5.82,1.40,2.82,1329.0,22.04
8,9.0,Moncloa-Aravaca,26039.0,5.37,1.35,2.06,1340.0,33.07
9,10.0,Latina,18974.0,8.52,2.04,4.84,1147.5,24.44


## Dataframe educación

### Extraer información del dataframe principal

In [33]:
educacion_indicadores = [ 
    'Número Habitantes', 'Escuelas Infantiles Públicas CAM',
    'Colegios Públicos Infantil y Primaria', 'Escuelas Infantiles Municipales',
    'Institutos Públicos de Educación Secundaria',
    'Casos trabajados por el programa de absentismo municipal', 
    'Población mayor/igual  de 25 años  que no sabe leer ni escribir o sin estudios', 
    'Población mayor/igual  de 25 años con enseñanza primaria incompleta', 
    'Población mayor/igual  de 25 años con Bachiller Elemental, Graduado Escolar, ESO, Formación profesional 1º grado',  
    'Población mayor/igual  de 25 años con Formación profesional 2º grado, Bachiller Superior o BUP', 
    'Población mayor/igual  de 25 años con titulación media, diplomatura, arquitectura o ingeniería técnica',  
    'Población mayor/igual  de 25 años  con estudios superiores, licenciatura, arquitectura, ingeniería sup., estudios sup. no universitarios, doctorado,  postgraduado']

# Crear dataframe de Educación por distritos
df_educacion = df[df['indicador_completo'].isin(educacion_indicadores)].pivot_table(
    index=['cod_distrito', 'distrito'], 
    columns='indicador_completo', 
    values='valor_indicador', 
    aggfunc='first').reset_index()

### Añadir información de centros educativos

In [34]:
# Cargamos el listado de centros educativos 
df_centros_educativos = pd.read_csv('../data/centros-educativos.csv', sep=';', encoding='latin1')

estandarizar_columnas(df_centros_educativos)

# Calcular el recuento de centros educativos por distrito
df_centros_educativos = df_centros_educativos.groupby('cod_distrito').size().reset_index(name='recuento_centros')

# Unir los dataframes
df_educacion = pd.merge(df_educacion, df_centros_educativos, on='cod_distrito', how='left')

### Limpieza y preparación

In [95]:
estandarizar_columnas(df_educacion)

In [36]:
# Tasa de centros educativos por cada 1000 habitantes
df_educacion['tasa_centros'] = ((df_educacion['recuento_centros'] / df_educacion['numero_habitantes']) * 1000).round(2)

# Tasa de centros públicos de enseñanza obligatoria por 1000 habitantes
df_educacion['tasa_centros_publicos_obligatoria'] = ((df_educacion['colegios_publicos_infantil_y_primaria'] +
                                                      df_educacion['escuelas_infantiles_municipales'] +
                                                      df_educacion['escuelas_infantiles_publicas_cam'] +
                                                      df_educacion['institutos_publicos_de_educacion_secundaria']) / df_educacion['numero_habitantes'] * 1000).round(2)

# Tasa de absentismo
df_educacion['tasa_absentismo'] = ((df_educacion['casos_trabajados_por_el_programa_de_absentismo_municipal'] / df_educacion['numero_habitantes']) *1000).round(2)

# Tasa de analfabetismo
df_educacion['tasa_analfabetismo'] = ((df_educacion['poblacion_mayor/igual__de_25_anos__que_no_sabe_leer_ni_escribir_o_sin_estudios'] / df_educacion['numero_habitantes'])*1000).round(2)

# Tasa de población con solo la enseñanza obligatoria
df_educacion['tasa_poblacion_educacion_obligatoria'] = ((df_educacion['poblacion_mayor/igual__de_25_anos_con_bachiller_elemental,_graduado_escolar,_eso,_formacion_profesional_1o_grado']
                                                        / df_educacion['numero_habitantes'])*1000).round(2)

# Tasa de población con educación superior
df_educacion['tasa_poblacion_educacion_superior'] = ((df_educacion['poblacion_mayor/igual__de_25_anos__con_estudios_superiores,_licenciatura,_arquitectura,_ingenieria_sup.,_estudios_sup._no_universitarios,_doctorado,__postgraduado']
                                                     / df_educacion['numero_habitantes'])*1000).round(2)

In [37]:
# Seleccionamos solo las columnas interesantes
df_educacion = df_educacion[['cod_distrito', 'distrito', 'tasa_centros', 'tasa_centros_publicos_obligatoria',
                            'tasa_absentismo', 'tasa_analfabetismo','tasa_poblacion_educacion_obligatoria', 'tasa_poblacion_educacion_superior']]

### Resultado

In [38]:
df_educacion

,cod_distrito,distrito,tasa_centros,tasa_centros_publicos_obligatoria,tasa_absentismo,tasa_analfabetismo,tasa_poblacion_educacion_obligatoria,tasa_poblacion_educacion_superior
0,1.0,Centro,0.64,0.19,0.68,25.64,160.54,360.07
1,2.0,Arganzuela,0.47,0.13,0.42,20.38,159.08,299.30
2,3.0,Retiro,0.53,0.10,0.21,15.44,126.71,355.26
3,4.0,Salamanca,0.45,0.05,0.17,12.93,115.96,400.37
4,5.0,Chamartín,0.87,0.13,0.18,11.52,108.79,395.50
5,6.0,Tetuán,0.34,0.14,1.43,30.92,212.76,251.80
6,7.0,Chamberí,0.68,0.12,0.22,13.47,122.75,411.50
7,8.0,Fuencarral-El Pardo,0.67,0.20,0.67,26.49,154.56,271.07
8,9.0,Moncloa-Aravaca,1.54,0.18,0.44,18.66,130.97,334.43
9,10.0,Latina,0.44,0.17,1.35,48.95,255.53,128.68


## Dataframe de cultura, deporte y ocio

### Extraer información del dataframe principal

In [39]:
cultura_indicadores = [
    'Grado de satisfacción con la organización de fiestas y eventos populares', 
    'Número Habitantes',
    'Bibliotecas públicas Municipales', 
    'Bibliotecas públicas Comunidad Madrid', 
    'Centros y Espacios Culturales', 
    'Centros deportivos Municipales', 
    'Superficie deportiva m2', 
    'Relación de Superficie de zonas verdes y Parques de distrito (ha) entre número de Habitantes *10.000', 
    'Superficie de Parques Históricos, Singulares y Forestales por distrito  entre número de Habitantes *10.000', 
    'Grado de satisfacción con los espacios verdes', 
    'Grado de satisfacción con los parques infantiles', 
    'Grado de satisfacción con los centros culturales', 
    'Grado de satisfacción con las instalaciones deportivas']

# Crear dataframe de Cultura, Deporte y Ocio por distritos
df_cultura = df[df['indicador_completo'].isin(cultura_indicadores)].pivot_table(
    index=['cod_distrito', 'distrito'], 
    columns='indicador_completo', 
    values='valor_indicador', 
    aggfunc='first').reset_index()

### Limpieza y preparación

In [41]:
# Tras revisar la base de datos, vemos que NaN
df_cultura.fillna(0, inplace=True)

estandarizar_columnas(df_cultura)

#Simplificar el nombre de las columnas
df_cultura.rename(columns={
    'grado_de_satisfaccion_con_la_organizacion_de_fiestas_y_eventos_populares': 'satisfaccion_fiestas_eventos',
    'grado_de_satisfaccion_con_los_espacios_verdes': 'satisfaccion_espacios_verdes',
    'grado_de_satisfaccion_con_las_instalaciones_deportivas':'satisfaccion_instalaciones_deportivas',
    'grado_de_satisfaccion_con_los_centros_culturales': 'satisfaccion_centros_culturales',
    'relacion_de_superficie_de_zonas_verdes_y_parques_de_distrito_(ha)_entre_numero_de_habitantes_*10.000': 'tasa_zonas_verdes'}, inplace=True)

# Tasa de bibliotecas públicas por 1000 habitantes
df_cultura['tasa_bibliotecas'] = ((df_cultura['bibliotecas_publicas_comunidad_madrid'] +
                                  df_cultura['bibliotecas_publicas_municipales']) / df_cultura['numero_habitantes']) * 10000

# Tasa de superficie deportiva por 10000 habitantes
df_cultura['tasa_superficie_deportiva'] = (df_cultura['superficie_deportiva_m2'] / df_cultura['numero_habitantes']) * 10000

# Tasa de centros culturales por 10000 habitantes
df_cultura['tasa_centros_culturales'] = (df_cultura['centros_y_espacios_culturales'] / df_cultura['numero_habitantes']) * 10000


# Filtrar columnas relevantes para el análisis
df_cultura = df_cultura[['cod_distrito', 'distrito', 'tasa_bibliotecas', 'tasa_centros_culturales',
                         'tasa_zonas_verdes', 'satisfaccion_espacios_verdes', 'tasa_superficie_deportiva',
                         'satisfaccion_instalaciones_deportivas', 'satisfaccion_fiestas_eventos']]

### Resultado

In [42]:
df_cultura

indicador_completo,cod_distrito,distrito,tasa_bibliotecas,tasa_centros_culturales,tasa_zonas_verdes,satisfaccion_espacios_verdes,tasa_superficie_deportiva,satisfaccion_instalaciones_deportivas,satisfaccion_fiestas_eventos
0,1.0,Centro,0.370697,0.444837,2.52,6.5,0.000000,5.5,6.9
1,2.0,Arganzuela,0.130014,0.260027,5.81,7.0,5088.994344,6.6,6.7
2,3.0,Retiro,0.167534,0.418834,3.09,7.6,5225.374647,7.4,6.9
3,4.0,Salamanca,0.136848,0.273695,2.43,6.4,344.513781,6.0,6.5
4,5.0,Chamartín,0.137113,0.137113,3.77,6.7,1014.636822,6.9,6.7
5,6.0,Tetuán,0.126633,0.253266,4.00,6.8,1162.172259,6.8,6.9
6,7.0,Chamberí,0.143423,0.071711,0.78,6.9,0.000000,6.9,6.9
7,8.0,Fuencarral-El Pardo,0.081294,0.325175,14.46,6.4,5229.309693,6.4,6.5
8,9.0,Moncloa-Aravaca,0.083736,0.418680,16.10,7.2,15447.191914,6.6,6.7
9,10.0,Latina,0.125969,0.377907,11.58,7.1,11180.664612,6.5,6.6


## Dataframe servicios sociales

### Extraer información del dataframe principal

In [43]:
servicios_sociales_indicadores = [
    'Tasa de riesgo de pobreza o exclusión social', 
    'Número Habitantes',
    'Personas atendidas en la Unidad de Primera Atención en Centros de Servicios Sociales', 
    'Personas con Servicio de Ayuda a Domicilio (modalidad auxiliar de hogar)',  
    'Personas socias de los Centros Municipales de Mayores', 
    'Demandas de intervención en los Centros de Atención a la Infancia (CAI)', 
    'Centros de Servicios Sociales', 
    'Centros Municipales de Mayores']

# Crear dataframe de Servicios Sociales por distritos
df_social = df[df['indicador_completo'].isin(servicios_sociales_indicadores)].pivot_table(
    index=['cod_distrito', 'distrito'], 
    columns='indicador_completo', 
    values='valor_indicador', 
    aggfunc='first').reset_index()

indicador_completo,cod_distrito,distrito,Centros Municipales de Mayores,Centros de Servicios Sociales,Demandas de intervención en los Centros de Atención a la Infancia (CAI),Número Habitantes,Personas atendidas en la Unidad de Primera Atención en Centros de Servicios Sociales,Personas con Servicio de Ayuda a Domicilio (modalidad auxiliar de hogar),Personas socias de los Centros Municipales de Mayores
0,1.0,Centro,4.0,2.0,81.0,134881.0,3869.0,2357.0,8200.0
1,2.0,Arganzuela,2.0,1.0,52.0,153830.0,4721.0,2209.0,16218.0
2,3.0,Retiro,3.0,1.0,49.0,119379.0,1718.0,1411.0,16089.0
3,4.0,Salamanca,2.0,1.0,42.0,146148.0,3475.0,1946.0,11334.0
4,5.0,Chamartín,3.0,2.0,40.0,145865.0,3383.0,1679.0,12279.0
5,6.0,Tetuán,3.0,2.0,140.0,157937.0,5105.0,3400.0,13719.0
6,7.0,Chamberí,2.0,1.0,59.0,139448.0,2797.0,2254.0,11412.0
7,8.0,Fuencarral-El Pardo,6.0,2.0,131.0,246021.0,2787.0,5159.0,29297.0
8,9.0,Moncloa-Aravaca,4.0,2.0,61.0,119423.0,2782.0,2269.0,12618.0
9,10.0,Latina,4.0,3.0,231.0,238154.0,7225.0,9542.0,24621.0


### Añadir datos residencias

In [44]:
df_residencias = pd.read_csv('../data/residencias_apartamentos_mayores.csv', sep=';', encoding='latin1')

estandarizar_columnas(df_residencias)

# Agrupar por distrito y contar el número de centros en cada distrito
df_residencias = df_residencias.groupby('cod_distrito').size().reset_index(name='recuento_residencias')

# Unir los dataframes
df_social = pd.merge(df_social, df_residencias, on='cod_distrito', how='left')

,cod_distrito,distrito,Centros Municipales de Mayores,Centros de Servicios Sociales,Demandas de intervención en los Centros de Atención a la Infancia (CAI),Número Habitantes,Personas atendidas en la Unidad de Primera Atención en Centros de Servicios Sociales,Personas con Servicio de Ayuda a Domicilio (modalidad auxiliar de hogar),Personas socias de los Centros Municipales de Mayores,recuento_residencias
0,1.0,Centro,4.0,2.0,81.0,134881.0,3869.0,2357.0,8200.0,4
1,2.0,Arganzuela,2.0,1.0,52.0,153830.0,4721.0,2209.0,16218.0,8
2,3.0,Retiro,3.0,1.0,49.0,119379.0,1718.0,1411.0,16089.0,5
3,4.0,Salamanca,2.0,1.0,42.0,146148.0,3475.0,1946.0,11334.0,10
4,5.0,Chamartín,3.0,2.0,40.0,145865.0,3383.0,1679.0,12279.0,15
5,6.0,Tetuán,3.0,2.0,140.0,157937.0,5105.0,3400.0,13719.0,4
6,7.0,Chamberí,2.0,1.0,59.0,139448.0,2797.0,2254.0,11412.0,6
7,8.0,Fuencarral-El Pardo,6.0,2.0,131.0,246021.0,2787.0,5159.0,29297.0,12
8,9.0,Moncloa-Aravaca,4.0,2.0,61.0,119423.0,2782.0,2269.0,12618.0,26
9,10.0,Latina,4.0,3.0,231.0,238154.0,7225.0,9542.0,24621.0,6


### Limpieza y preparación de datos

In [45]:
#Simplificar nombres de columnas
df_social.rename(columns={
    'Demandas de intervención en los Centros de Atención a la Infancia (CAI)': 'demandas_cai',
    'Personas atendidas en la Unidad de Primera Atención en Centros de Servicios Sociales': 'personas_atendidas_ss',
    'Personas con Servicio de Ayuda a Domicilio (modalidad auxiliar de hogar)': 'personas_ayuda_domicilio'}, inplace=True)

#Estandarizar el resto de columnas
estandarizar_columnas(df_social)

In [ ]:
# Tasa de demandas a Servicios Sociales por la infancia
df_social['tasa_demandas_cai'] = (df_social['demandas_cai'] / df_social['numero_habitantes']) * 1000

# Tasa de personas atendidas por los Servicios Sociales
df_social['tasa_personas_atendidas_ss'] = (df_social['personas_atendidas_ss'] / df_social['numero_habitantes']) * 1000

# Tasa de ayuda a domicilio
df_social['tasa_ayuda_domicilio'] = (df_social['personas_ayuda_domicilio'] / df_social['numero_habitantes']) * 1000

# Tasa de residencias, apartamentos y centros de día para mayores
df_social['tasa_residencias'] = (df_social['recuento_residencias'] / df_social['numero_habitantes']) * 1000

In [48]:
# Tasa de centros de Servicios Sociales
df_social['tasa_centros_ss'] = (df_social['centros_de_servicios_sociales'] / df_social['numero_habitantes']) * 1000

In [50]:
#Seleccionar solo las columnas relevantes

df_social = df_social[['tasa_centros_ss','tasa_demandas_cai', 'tasa_personas_atendidas_ss', 'tasa_ayuda_domicilio', 'tasa_residencias']]

### Resultado

In [51]:
df_social

,tasa_centros_ss,tasa_demandas_cai,tasa_personas_atendidas_ss,tasa_ayuda_domicilio,tasa_residencias
0,0.014828,0.600529,28.684544,17.474663,0.029656
1,0.006501,0.338035,30.689722,14.360008,0.052005
2,0.008377,0.410457,14.391141,11.819499,0.041883
3,0.006842,0.287380,23.777267,13.315269,0.068424
4,0.013711,0.274226,23.192678,11.510643,0.102835
5,0.012663,0.886429,32.323015,21.527571,0.025327
6,0.007171,0.423097,20.057656,16.163731,0.043027
7,0.008129,0.532475,11.328301,20.969755,0.048776
8,0.016747,0.510789,23.295345,18.999690,0.217714
9,0.012597,0.969961,30.337513,40.066512,0.025194


## Dataframe presupuesto

In [90]:
# Definir la ruta de la carpeta donde están los CSV
ruta_carpeta = '../data/presupuestos'

# Procesar los archivos CSV
df_inversiones = procesar_presupuestos(ruta_carpeta)

# Convertir a float el código de distrito
df_inversiones['cod_distrito'].astype(float)

In [94]:
df_inversiones

,cod_distrito,año,area_inversion,total_invertido
0,01,2012,Cultura,6220891.60
1,01,2012,"Deportes, juventud y esparcimiento",27813722.14
2,01,2012,Educación,2828895.36
3,01,2012,Inversiones de carácter general,344577.32
4,01,2012,Mantenimiento urbano,3207506.34
...,...,...,...,...
2641,21,2022,Medio ambiente,645442.22
2642,21,2022,Otros bienes públicos de carácter social,14207.82
2643,21,2022,Protección Civil y seguridad ciudadana,63172.34
2644,21,2022,Protección y promoción social,105930.74
